In [2]:
%matplotlib inline
import csv
import numpy as np
import pandas as pd
from scipy.signal import *
from numpy.fft import *
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 15}) # pour augmenter la taille de caractère dans les graphiques
from ipywidgets import * # Pour insérer des widgets (éléments graphiques) dans le notebook
# Attention! Installer scikit-learn pour avoir accès aux algorithmes de classification
# Installer version 0.20 au plus récente (0.22.1) pour avoir accès à la nouvelle fonction tree.plot_tree()
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier # Méthode KNN
from sklearn.naive_bayes import GaussianNB
from scipy.ndimage import interpolation

In [115]:
def import_raw(step_type, file_nb=0, plot_fig=False, save_fig=False):
    path_raw  =  'etapdance2' + '/' + step_type + '/' + step_type + '_full_' + str(file_nb) + '.txt'
    data_raw_df = pd.read_csv(path_raw)
    time = np.array(data_raw_df.iloc[:,0])
    if plot_fig:
        # Gyroscope 0 et 1 (x,y,z)
        plt.figure(figsize=(15,6))
        for n in range(1,len(data_raw_df.columns)-9):# pour récupérer juste les Gyro0
            plt.plot(time,np.array(data_raw_df.iloc[:,n]),label=data_raw_df.columns[n])
        for n in range(7,len(data_raw_df.columns)-3):# pour récupérer juste les Gyro1
            plt.plot(time,np.array(data_raw_df.iloc[:,n]),label=data_raw_df.columns[n])
        plt.ylabel('amplitude (deg/sec)')
        plt.xlabel('échantillons (-)')
        plt.autoscale(axis='x',tight=True);
        plt.legend()
        if save_fig:
            plt.savefig(fname='data_raw_' + 'gyro_' + step_type +'.png')
        plt.show()

        # Accéléromètre 0 et 1 (x,y,z)   
        plt.figure(figsize=(15,6))
        for n in range(4,len(data_raw_df.columns)-6):# pour récupérer juste les Accel0
            plt.plot(time,np.array(data_raw_df.iloc[:,n]),label=data_raw_df.columns[n])
        for n in range(10,len(data_raw_df.columns)):# pour récupérer juste les Accel1
            plt.plot(time,np.array(data_raw_df.iloc[:,n]),label=data_raw_df.columns[n])
        plt.ylabel('amplitude (g)')
        plt.xlabel('échantillons (-)')
        plt.autoscale(axis='x',tight=True);
        plt.legend()
        if save_fig:
            plt.savefig(fname='data_raw_' + 'accel_' + step_type +'.png')
        plt.show()
    return data_raw_df

def STE_param(data_raw_df,f_ech, SecPerFrame):
    data_length = len(data_raw_df)
    sampsPerSec = f_ech
    sampsPerFrame = int(sampsPerSec * SecPerFrame)
    nFrames = int(data_length / sampsPerFrame)    
    return sampsPerFrame, nFrames

def STE_process(data_raw_df, sampsPerFrame, nFrames):
    STEs = []
    STEs_df = pd.Dataframe()
    for k in range(nFrames):
        startIdx = k * sampsPerFrame
        stopIdx = startIdx + sampsPerFrame
        window = np.zeros(data_raw_df.iloc[:,0].shape)
        window[startIdx:stopIdx] = 1 # rectangular window
        for n in range(1,len(data_raw_df.columns)-9):
            STE = sum((data_raw_df.iloc[:,n] ** 2) * (window ** 2))
            STEs.append(STE)
        
    return STEs

In [114]:
raw_df = import_raw(step_type='step')
samp_per_frame, n_frames = STE_param(raw_df,f_ech=91, SecPerFrame =0.09)
pouet = STE_process(raw_df,samp_per_frame, n_frames)